In [1]:
import os
os.chdir("1_feature_engineering")
exec(open("extract_review_features.py").read())

/home/richard/anaconda3/envs/insight/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [2]:
feature_matrix = produce_feature_matrix(data2)

Preprocessing Stem & Join (P): 3.2486469745635986
Preprocessing No Stem (P): 2.957641839981079
Preprocessing No Stem & Punct (P): 2.969756603240967
TFIDF: 0.07323431968688965
Topics: 0.3993186950683594


/home/richard/anaconda3/envs/insight/lib/python3.7/site-packages/ipykernel_launcher.py:498: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Metadata: 0.6300594806671143


/home/richard/anaconda3/envs/insight/lib/python3.7/site-packages/ipykernel_launcher.py:354: RuntimeWarning: invalid value encountered in double_scalars


Syntax: 32.44103169441223
Embeddings: 82.87586951255798


In [3]:
feature_matrix.shape

(1050, 459)

#### Split out features by categories

In [4]:
feature_names = feature_matrix.columns[1:]

In [5]:
metadata_columns = feature_names[:5]

In [6]:
syntax_columns = feature_names[5:149]

In [7]:
syntax_columns[:50]

Index(['NOUN', 'VERB', 'ADJ', 'ADV', '.', 'ADP', 'NUM', 'PRON', 'PRT', 'DET',
       'CONJ', 'X', 'NOUN_VERB', 'NOUN_ADJ', 'NOUN_ADV', 'NOUN_.', 'NOUN_ADP',
       'NOUN_NUM', 'NOUN_PRON', 'NOUN_PRT', 'NOUN_DET', 'NOUN_CONJ', 'NOUN_X',
       'VERB_NOUN', 'VERB_ADJ', 'VERB_ADV', 'VERB_.', 'VERB_ADP', 'VERB_NUM',
       'VERB_PRON', 'VERB_PRT', 'VERB_DET', 'VERB_CONJ', 'VERB_X', 'ADJ_NOUN',
       'ADJ_VERB', 'ADJ_ADV', 'ADJ_.', 'ADJ_ADP', 'ADJ_NUM', 'ADJ_PRON',
       'ADJ_PRT', 'ADJ_DET', 'ADJ_CONJ', 'ADJ_X', 'ADV_NOUN', 'ADV_VERB',
       'ADV_ADJ', 'ADV_.', 'ADV_ADP'],
      dtype='object')

In [8]:
embeddings_columns = feature_names[149:158]

In [9]:
tfidf_unigrams_columns = feature_names[158:258]

In [10]:
tfidf_unigrams_columns[:50]

Index(['also', 'alway', 'amaz', 'around', 'ask', 'back', 'bar', 'best',
       'better', 'bit', 'buffet', 'burger', 'came', 'chees', 'chicken', 'come',
       'could', 'day', 'definit', 'delici', 'dessert', 'dinner', 'dish',
       'drink', 'eat', 'enjoy', 'even', 'everyth', 'experi', 'favorit',
       'first', 'flavor', 'food', 'fresh', 'fri', 'friend', 'get', 'go',
       'good', 'got', 'great', 'hour', 'know', 'like', 'littl', 'locat',
       'look', 'lot', 'love', 'made'],
      dtype='object')

In [11]:
tfidf_bigrams_columns = feature_names[258:358]

In [12]:
tfidf_bigrams_columns[:50]

Index(['10 minut', '15 minut', '20 minut', 'also order', 'back tri',
       'best part', 'ca wait', 'came back', 'chicken waffl', 'come back',
       'cook perfectli', 'crab leg', 'custom servic', 'definit back',
       'definit come', 'definit worth', 'dim sum', 'dine experi',
       'even though', 'everi time', 'feel like', 'felt like', 'first time',
       'foie gra', 'food came', 'food good', 'food great', 'food servic',
       'french toast', 'fri chicken', 'fri rice', 'gave us', 'give place',
       'go back', 'go wrong', 'good food', 'good servic', 'good thing',
       'great food', 'great place', 'great servic', 'happi hour',
       'highli recommend', 'ice cream', 'la vega', 'last night', 'last time',
       'late night', 'like place', 'littl bit'],
      dtype='object')

In [13]:
topics_columns = feature_names[358:458]

#### Get the validation data

In [21]:
from sklearn.ensemble import RandomForestClassifier
import pickle
import copy

In [22]:
clf = RandomForestClassifier(n_estimators=1000,
                             max_depth=3,
                             min_samples_leaf=10,
                             max_features=0.6,
                             random_state=10191994,
                             class_weight="balanced",
                             n_jobs=-1)

In [42]:
X_train_ambiance = pickle.load(open("../../0_data/5_train_validation_test/X_train_ambiance_1050.pickle", "rb"))
y_train_ambiance = pickle.load(open("../../0_data/5_train_validation_test/y_train_ambiance_1050.pickle", "rb"))
X_validation_ambiance = pickle.load(open("../../0_data/5_train_validation_test/X_validation_ambiance_1050.pickle", "rb"))
y_validation_ambiance = pickle.load(open("../../0_data/5_train_validation_test/y_validation_ambiance_1050.pickle", "rb"))
X_test_ambiance = pickle.load(open("../../0_data/5_train_validation_test/X_test_ambiance_1050.pickle", "rb"))
y_test_ambiance = pickle.load(open("../../0_data/5_train_validation_test/y_test_ambiance_1050.pickle", "rb"))

In [70]:
def test_model(model, cat_list, train_x, train_y, test_x, test_y):
    train_x_df = pd.DataFrame(train_x)
    train_x_df.columns = ["text"] + list(feature_names)
    test_x_df = pd.DataFrame(test_x)
    test_x_df.columns = ["text"] + list(feature_names)
    
    tmp_train_df = pd.concat([train_x_df[x] for x in cat_list], axis=1)
#     print(tmp_train_df.columns)
    tmp_test_df = pd.concat([test_x_df[x] for x in cat_list], axis=1)
    
    tmp_model = copy.deepcopy(model)
    tmp_model.fit(tmp_train_df, train_y)
    
#     return tmp_model
    return (tmp_model, tmp_model.score(
        tmp_test_df, test_y), tmp_test_df)

In [71]:
data.describe()

,food,service,price,ambiance
count,1050.000000,1050.000000,1050.000000,1050.000000
mean,0.908571,0.637143,0.263810,0.404762
std,0.288355,0.481053,0.440907,0.491080
min,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,0.000000
50%,1.000000,1.000000,0.000000,0.000000
75%,1.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000


In [72]:
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix

In [74]:
ambiance_clf1, ambiance_syntax, ambiance_test1 = test_model(clf, [syntax_columns], X_train_ambiance, y_train_ambiance, X_test_ambiance, y_test_ambiance)
ambiance_clf2, ambiance_metadata, ambiance_test2 = test_model(clf, [metadata_columns, syntax_columns], X_train_ambiance, y_train_ambiance, X_test_ambiance, y_test_ambiance)
ambiance_clf3, ambiance_tfidf, ambiance_test3 = test_model(clf, [metadata_columns, syntax_columns, tfidf_unigrams_columns, tfidf_bigrams_columns], X_train_ambiance, y_train_ambiance, X_test_ambiance, y_test_ambiance)
ambiance_clf4, ambiance_topics, ambiance_test4 = test_model(clf, [metadata_columns, syntax_columns, tfidf_unigrams_columns, tfidf_bigrams_columns, topics_columns], X_train_ambiance, y_train_ambiance, X_test_ambiance, y_test_ambiance)
ambiance_clf5, ambiance_embeddings, ambiance_test5 = test_model(clf, [metadata_columns, syntax_columns, tfidf_unigrams_columns, tfidf_bigrams_columns, topics_columns, embeddings_columns], X_train_ambiance, y_train_ambiance, X_test_ambiance, y_test_ambiance)
ambiance_p = pickle.load(open("../../4_models/rf_ambiance_4iterations_athreshold_5000_200_1050.pickle", "rb"))
ambiance_pseudolabel = ambiance_p.score(X_test_ambiance[:, 1:], y_test_ambiance)

/home/richard/anaconda3/envs/insight/lib/python3.7/site-packages/ipykernel_launcher.py:12: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  if sys.path[0] == '':
/home/richard/anaconda3/envs/insight/lib/python3.7/site-packages/ipykernel_launcher.py:12: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  if sys.path[0] == '':
/home/richard/anaconda3/envs/insight/lib/python3.7/site-packages/ipykernel_launcher.py:12: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  if sys.path[0] == '':
/home/richard/anaconda3/envs/insight/lib/python3.7/site-packages/ipykernel_launcher.py:12: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the

In [28]:
confusion_matrix(y_test_ambiance, ambiance_p.predict(X_test_ambiance[:, 1:]))

array([[106,  19],
       [ 25,  60]])

In [29]:
precision_recall_fscore_support(y_test_ambiance, ambiance_p.predict(X_test_ambiance[:, 1:]))

(array([0.80916031, 0.75949367]),
 array([0.848     , 0.70588235]),
 array([0.828125  , 0.73170732]),
 array([125,  85]))

In [30]:
ambiance = [ambiance_syntax, ambiance_metadata, ambiance_tfidf, ambiance_topics, ambiance_embeddings, ambiance_pseudolabel]

In [75]:
ambiance_clf5.score(ambiance_test5, y_test_ambiance)

0.780952380952381

In [76]:
ambiance_test5.columns

Index(['sen_len', 'sen_avg_len', 'sen_med_len', 'sen_money_len', 'sen_num_len',
       'NOUN', 'VERB', 'ADJ', 'ADV', '.',
       ...
       'topic_99', 'food_3avg_sim', 'delicious_3avg_sim', 'service_3avg_sim',
       'staff_3avg_sim', 'price_3avg_similarity', 'cost_3avg_similarity',
       'ambiance_3avg_similarity', 'patio_3avg_similarity',
       'loud_3avg_similarity'],
      dtype='object', length=458)

In [77]:
ambiance

[0.5857142857142857,
 0.5857142857142857,
 0.6238095238095238,
 0.7095238095238096,
 0.780952380952381,
 0.7904761904761904]

In [ ]:
X_train_service = pickle.load(open("../../0_data/5_train_validation_test/X_train_service_1050.pickle", "rb"))
y_train_service = pickle.load(open("../../0_data/5_train_validation_test/y_train_service_1050.pickle", "rb"))
X_validation_service = pickle.load(open("../../0_data/5_train_validation_test/X_validation_service_1050.pickle", "rb"))
y_validation_service = pickle.load(open("../../0_data/5_train_validation_test/y_validation_service_1050.pickle", "rb"))
X_test_service = pickle.load(open("../../0_data/5_train_validation_test/X_test_service_1050.pickle", "rb"))
y_test_service = pickle.load(open("../../0_data/5_train_validation_test/y_test_service_1050.pickle", "rb"))

service_clf1, service_syntax = test_model(clf, [syntax_columns], X_train_service, y_train_service, X_test_service, y_test_service)
service_clf2, service_metadata = test_model(clf, [metadata_columns, syntax_columns], X_train_service, y_train_service, X_test_service, y_test_service)
service_clf3, service_tfidf = test_model(clf, [metadata_columns, syntax_columns, tfidf_unigrams_columns, tfidf_bigrams_columns], X_train_service, y_train_service, X_test_service, y_test_service)
service_clf4, service_topics = test_model(clf, [metadata_columns, syntax_columns, tfidf_unigrams_columns, tfidf_bigrams_columns, topics_columns], X_train_service, y_train_service, X_test_service, y_test_service)
service_clf5, service_embeddings = test_model(clf, [metadata_columns, syntax_columns, tfidf_unigrams_columns, tfidf_bigrams_columns, topics_columns, embeddings_columns], X_train_service, y_train_service, X_test_service, y_test_service)
service_p = pickle.load(open("../../4_models/rf_service_4iterations_athreshold_5000_200_1050.pickle", "rb"))
service_pseudolabel = service_p.score(X_test_service[:, 1:], y_test_service)

service = [service_syntax, service_metadata, service_tfidf, service_topics, service_embeddings, service_pseudolabel]

In [ ]:
precision_recall_fscore_support(y_test_service, service_p.predict(X_test_service[:, 1:]))

In [ ]:
service_clf5.score(X_test_service[:, 1:], y_test_service)

In [ ]:
service

In [ ]:
X_train_price = pickle.load(open("../../0_data/5_train_validation_test/X_train_price_1050.pickle", "rb"))
y_train_price = pickle.load(open("../../0_data/5_train_validation_test/y_train_price_1050.pickle", "rb"))
X_validation_price = pickle.load(open("../../0_data/5_train_validation_test/X_validation_price_1050.pickle", "rb"))
y_validation_price = pickle.load(open("../../0_data/5_train_validation_test/y_validation_price_1050.pickle", "rb"))
X_test_price = pickle.load(open("../../0_data/5_train_validation_test/X_test_price_1050.pickle", "rb"))
y_test_price = pickle.load(open("../../0_data/5_train_validation_test/y_test_price_1050.pickle", "rb"))

price_clf1, price_syntax = test_model(clf, [syntax_columns], X_train_price, y_train_price, X_test_price, y_test_price)
price_clf2, price_metadata = test_model(clf, [metadata_columns, syntax_columns], X_train_price, y_train_price, X_test_price, y_test_price)
price_clf3, price_tfidf = test_model(clf, [metadata_columns, syntax_columns, tfidf_unigrams_columns, tfidf_bigrams_columns], X_train_price, y_train_price, X_test_price, y_test_price)
price_clf4, price_topics = test_model(clf, [metadata_columns, syntax_columns, tfidf_unigrams_columns, tfidf_bigrams_columns, topics_columns], X_train_price, y_train_price, X_test_price, y_test_price)
price_clf5, price_embeddings = test_model(clf, [metadata_columns, syntax_columns, tfidf_unigrams_columns, tfidf_bigrams_columns, topics_columns, embeddings_columns], X_train_price, y_train_price, X_test_price, y_test_price)
price_p = pickle.load(open("../../4_models/rf_price_4iterations_athreshold_5000_200_1050.pickle", "rb"))
price_pseudolabel = price_p.score(X_test_price[:, 1:], y_test_price)

price = [price_syntax, price_metadata, price_tfidf, price_topics, price_embeddings, price_pseudolabel]

In [ ]:
confusion_matrix(y_test_price, price_p.predict(X_test_price[:, 1:]))

In [ ]:
45/68

In [ ]:
precision_recall_fscore_support(y_test_price, price_p.predict(X_test_price[:, 1:]))

In [ ]:
price

In [ ]:
X_train_food = pickle.load(open("../../0_data/5_train_validation_test/X_train_food_1050.pickle", "rb"))
y_train_food = pickle.load(open("../../0_data/5_train_validation_test/y_train_food_1050.pickle", "rb"))
X_validation_food = pickle.load(open("../../0_data/5_train_validation_test/X_validation_food_1050.pickle", "rb"))
y_validation_food = pickle.load(open("../../0_data/5_train_validation_test/y_validation_food_1050.pickle", "rb"))
X_test_food = pickle.load(open("../../0_data/5_train_validation_test/X_test_food_1050.pickle", "rb"))
y_test_food = pickle.load(open("../../0_data/5_train_validation_test/y_test_food_1050.pickle", "rb"))

food_clf1, food_syntax = test_model(clf, [syntax_columns], X_train_food, y_train_food, X_test_food, y_test_food)
food_clf2, food_metadata = test_model(clf, [metadata_columns, syntax_columns], X_train_food, y_train_food, X_test_food, y_test_food)
food_clf3, food_tfidf = test_model(clf, [metadata_columns, syntax_columns, tfidf_unigrams_columns, tfidf_bigrams_columns], X_train_food, y_train_food, X_test_food, y_test_food)
food_clf4, food_topics = test_model(clf, [metadata_columns, syntax_columns, tfidf_unigrams_columns, tfidf_bigrams_columns, topics_columns], X_train_food, y_train_food, X_test_food, y_test_food)
food_clf5, food_embeddings = test_model(clf, [metadata_columns, syntax_columns, tfidf_unigrams_columns, tfidf_bigrams_columns, topics_columns, embeddings_columns], X_train_food, y_train_food, X_test_food, y_test_food)
food_p = pickle.load(open("../../4_models/rf_food_4iterations_athreshold_5000_200_1050.pickle", "rb"))
food_pseudolabel = food_p.score(X_test_food[:, 1:], y_test_food)

food = [food_syntax, food_metadata, food_tfidf, food_topics, food_embeddings, food_pseudolabel]

In [ ]:
confusion_matrix(y_test_food, food_p.predict(X_test_food[:, 1:]))

In [ ]:
precision_recall_fscore_support(y_test_food, food_p.predict(X_test_food[:, 1:]))

In [ ]:
food

In [ ]:
df = pd.DataFrame({"features": ["1 [Syntax]",
                                "2 [+ Metadata]",
                                "3 [+ TF-IDF]",
                                "4 [+ Topics]",
                                "5 [+ Embeddings]",
                                "6 [Pseudo-Label]"],
                   "food": food,
                   "service": service,
                   "price": price,
                   "ambiance": ambiance})

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
f, ax = plt.subplots()
sns.lineplot(x="features", y="food", markers=True, data=df,
             color="blue", label="food")
sns.lineplot(x="features", y="service", markers=True, data=df,
             color="red", label="service")
sns.lineplot(x="features", y="price", markers=True, data=df,
             color="green", label="price")
sns.lineplot(x="features", y="ambiance", markers=True, data=df,
             color="orange", label="ambiance")
plt.setp(ax.get_xticklabels(), rotation=30)
ax.set(ylabel="Accuracy", xlabel="Model", ylim=(0.4, 1))
ax.legend(ncol=2, loc="lower right", frameon=True)
ax.grid(True)
sns.despine(left=True, bottom=True)
plt.tight_layout()
plt.savefig("../../3_reports/figures/model_accuracy_20190620.eps", format='eps', dpi=1000)

In [ ]:
f, ax = plt.subplots()
sns.lineplot(x="features", y="food", markers=True, data=df,
             color="blue", label="food")
plt.setp(ax.get_xticklabels(), rotation=30)
ax.set(ylabel="Accuracy", xlabel="Model", ylim=(0.4, 1))
ax.legend(ncol=2, loc="lower right", frameon=True)
ax.grid(True)
sns.despine(left=True, bottom=True)
plt.tight_layout()
plt.savefig("../../3_reports/figures/food_model_accuracy_20190620.eps", format='eps', dpi=1000)

In [ ]:
f, ax = plt.subplots()
sns.lineplot(x="features", y="service", markers=True, data=df,
             color="red", label="service")
plt.setp(ax.get_xticklabels(), rotation=30)
ax.set(ylabel="Accuracy", xlabel="Model", ylim=(0.4, 1))
ax.legend(ncol=2, loc="lower right", frameon=True)
ax.grid(True)
sns.despine(left=True, bottom=True)
plt.tight_layout()
plt.savefig("../../3_reports/figures/service_model_accuracy_20190620.eps", format='eps', dpi=1000)

In [ ]:
f, ax = plt.subplots()
sns.lineplot(x="features", y="price", markers=True, data=df,
             color="green", label="price")
plt.setp(ax.get_xticklabels(), rotation=30)
ax.set(ylabel="Accuracy", xlabel="Model", ylim=(0.4, 1))
ax.legend(ncol=2, loc="lower right", frameon=True)
ax.grid(True)
sns.despine(left=True, bottom=True)
plt.tight_layout()
plt.savefig("../../3_reports/figures/price_model_accuracy_20190620.eps", format='eps', dpi=1000)

In [ ]:
f, ax = plt.subplots()
sns.lineplot(x="features", y="ambiance", markers=True, data=df,
             color="orange", label="ambiance")
plt.setp(ax.get_xticklabels(), rotation=30)
ax.set(ylabel="Accuracy", xlabel="Model", ylim=(0.4, 1))
ax.legend(ncol=2, loc="lower right", frameon=True)
ax.grid(True)
sns.despine(left=True, bottom=True)
plt.tight_layout()
plt.savefig("../../3_reports/figures/ambiance_model_accuracy_20190620.eps", format='eps', dpi=1000)

#### Features

In [78]:
ambiance_embeddings_df = pd.DataFrame(sorted(list(zip(ambiance_test.columns, ambiance_clf5.feature_importances_)), key=lambda x: -x[1]))
ambiance_embeddings_df.columns = ["feature", "importance"]
ambiance_embeddings_tmp_df = ambiance_embeddings_df.iloc[:20, :]
ambiance_embeddings_tmp_df

,feature,importance
0,ambiance_3avg_similarity,0.440235
1,patio_3avg_similarity,0.274558
2,loud_3avg_similarity,0.141364
3,topic_0,0.025897
4,topic_8,0.005998
5,topic_25,0.005656
6,say,0.005603
7,topic_30,0.005133
8,staff_3avg_sim,0.004355
9,topic_22,0.003969


In [79]:
ambiance_p_df = pd.DataFrame(sorted(list(zip(feature_names, ambiance_p.feature_importances_)), key=lambda x: -x[1]))
ambiance_p_df.columns = ["feature", "importance"]

In [80]:
ambiance_p_tmp_df = ambiance_p_df.iloc[:6,:]

In [81]:
ambiance_p_tmp_df

,feature,importance
0,ambiance_3avg_similarity,0.511294
1,patio_3avg_similarity,0.216590
2,loud_3avg_similarity,0.141713
3,topic_0,0.034299
4,say,0.006216
5,topic_8,0.005607


In [ ]:
ambiance_p_tmp_df["feature_name"] = ["Sim. to Ambiance",
                                     "Sim. to Patio",
                                     "Sim. to Loud",
                                     "Topic: Atmosphere",
                                     "Say",
                                     "Topic: Interior"]

In [ ]:
f, ax = plt.subplots()
ax = sns.barplot(y="feature_name", x="importance", data=ambiance_p_tmp_df,
                 palette=["orange", "orange", "orange", "lightgray", "lightgray", "lightgray"])
ax.set(xlabel="Importance", ylabel="Feature", xlim=(0, 0.6), title="Top Six Features")
plt.tight_layout()
plt.savefig("../../3_reports/figures/ambiance_fi_20190620_embeddings.eps", format="eps", dpi=1000)

In [ ]:
service_p_df = pd.DataFrame(sorted(list(zip(feature_names, service_p.feature_importances_)), key=lambda x: -x[1]))
service_p_df.columns = ["feature", "importance"]
service_p_tmp_df = service_p_df.iloc[:6, :]
service_p_tmp_df["feature_name"] = ["Sim. to Service",
                                    "Sim. to Staff",
                                    "Servic",
                                    "Topic: Order",
                                    "Topic: Waitress",
                                    "Topic: Wait"]

In [ ]:
f, ax = plt.subplots()
ax = sns.barplot(y="feature_name", x="importance", data=service_p_tmp_df,
                 palette=["lightgray", "lightgray", "lightgray", "lightgray", "lightgray", "lightgray"])
ax.set(xlabel="Importance", ylabel="Feature", xlim=(0, 0.6), title="Top Six Features")
plt.tight_layout()
plt.savefig("../../3_reports/figures/service_fi_20190620_gray.eps", format="eps", dpi=1000)

In [ ]:
price_p_df = pd.DataFrame(sorted(list(zip(feature_names, price_p.feature_importances_)), key=lambda x: -x[1]))
price_p_df.columns = ["feature", "importance"]
price_p_tmp_df = price_p_df.iloc[:6, :]
price_p_tmp_df["feature_name"] = ["Sim. to Price",
                                  "Sim. to Cost",
                                  "Price",
                                  "(NUM, ADP)",
                                  "(NUM, PRT)",
                                  "$%"]

In [ ]:
f, ax = plt.subplots()
ax = sns.barplot(y="feature_name", x="importance", data=price_p_tmp_df,
                 palette=["lightgray", "lightgray", "lightgray", "green", "green", "green"])
ax.set(xlabel="Importance", ylabel="Feature", xlim=(0, 0.6), title="Top Six Features")
plt.tight_layout()
plt.savefig("../../3_reports/figures/price_fi_20190620_num_symbol.eps", format="eps", dpi=1000)

In [ ]:
food_p_df = pd.DataFrame(sorted(list(zip(feature_names, food_p.feature_importances_)), key=lambda x: -x[1]))
food_p_df.columns = ["feature", "importance"]
food_p_tmp_df = food_p_df.iloc[:6, :]
food_p_tmp_df["feature_name"] = ["Sim. to Delicious",
                                 "Sim. to Food",
                                 "Topic: Hotel",
                                 "Food",
                                 "Much",
                                 "Topic: Room"]

In [ ]:
f, ax = plt.subplots()
ax = sns.barplot(y="feature_name", x="importance", data=food_p_tmp_df,
                 palette=["blue", "blue", "lightgray", "lightgray", "lightgray", "lightgray"])
ax.set(xlabel="Importance", ylabel="Feature", xlim=(0, 0.6), title="Top Six Features")
plt.tight_layout()
plt.savefig("../../3_reports/figures/food_fi_20190620_embeddings.eps", format="eps", dpi=1000)

Percentage increase from pseudolabel

In [ ]:
food[-1]/food[-2]

In [ ]:
service[-1]/service[-2]

In [ ]:
price[-1]/price[-2]

In [ ]:
ambiance[-1]/ambiance[-2]

#### Random assignment accuracy

In [ ]:
data.columns

In [ ]:
from sklearn.metrics import confusion_matrix
import random
import copy
import numpy as np

In [ ]:
cm = confusion_matrix(y_test_ambiance, ambiance_p.predict(X_test_ambiance[:, 1:]))

In [ ]:
cm

In [ ]:
(cm[0][0] + cm[1][1])/np.sum(cm)

In [ ]:
166/210

In [ ]:
np.random.randint(2, size=100)

In [ ]:
def random_base(n, test):
    random.seed(10191994)
    tmp_list = []
    for i in range(n):
#         tmp = copy.deepcopy(test)
#         random.shuffle(tmp)
        tmp = np.random.randint(2, size=test.shape[0])
        tmp_cm = confusion_matrix(test, tmp)
        tmp_list.append((tmp_cm[0][0] + tmp_cm[1][1])/np.sum(tmp_cm))
    return np.mean(tmp_list)

In [ ]:
test0 = random_base(7000, y_test_ambiance)

In [ ]:
test0

In [ ]:
ambiance[-2]

In [ ]:
test1 = random_base(7000, y_test_price)

In [ ]:
test1

In [ ]:
price[-2]

In [ ]:
test2 = random_base(7000, y_test_service)

In [ ]:
test2

In [ ]:
service[-2]

In [ ]:
test3 = random_base(7000, y_test_food)

In [ ]:
test3

In [ ]:
food[-2]

In [ ]:
ambiance

In [ ]:
service

In [ ]:
price

In [ ]:
food

#### ROC AUC

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
food_auc = roc_auc_score(y_test_food, food_p.predict(X_test_food[:, 1:]))
service_auc = roc_auc_score(y_test_service, service_p.predict(X_test_service[:, 1:]))
price_auc = roc_auc_score(y_test_price, price_p.predict(X_test_price[:, 1:]))
ambiance_auc = roc_auc_score(y_test_ambiance, ambiance_p.predict(X_test_ambiance[:, 1:]))

In [ ]:
food_auc

In [ ]:
service_auc

In [ ]:
price_auc

In [ ]:
ambiance_auc

In [ ]:
print(roc_auc_score(y_validation_food, food_p.predict(X_validation_food[:, 1:])))
print(roc_auc_score(y_validation_service, service_p.predict(X_validation_service[:, 1:])))
print(roc_auc_score(y_validation_price, price_p.predict(X_validation_price[:, 1:])))
print(roc_auc_score(y_validation_ambiance, ambiance_p.predict(X_validation_ambiance[:, 1:])))

In [ ]:
print(roc_auc_score(y_test_food, food_p.predict(X_test_food[:, 1:])))
print(roc_auc_score(y_test_service, service_p.predict(X_test_service[:, 1:])))
print(roc_auc_score(y_test_price, price_p.predict(X_test_price[:, 1:])))
print(roc_auc_score(y_test_ambiance, ambiance_p.predict(X_test_ambiance[:, 1:])))

In [ ]:
def test_model2(model, cat_list, train_x, train_y, test_x, test_y):
    train_x_df = pd.DataFrame(train_x)
    train_x_df.columns = ["text"] + list(feature_names)
    test_x_df = pd.DataFrame(test_x)
    test_x_df.columns = ["text"] + list(feature_names)
    
    tmp_train_df = pd.concat([train_x_df[x] for x in cat_list], axis=1)
#     print(tmp_train_df.columns)
    tmp_test_df = pd.concat([test_x_df[x] for x in cat_list], axis=1)
    
    model.fit(tmp_train_df, train_y)
    
    return roc_auc_score(test_y, model.predict(tmp_test_df))

In [ ]:
print(test_model2(clf, [metadata_columns, syntax_columns, tfidf_unigrams_columns, tfidf_bigrams_columns, topics_columns, embeddings_columns], X_train_food, y_train_food, X_validation_food, y_validation_food))
print(test_model2(clf, [metadata_columns, syntax_columns, tfidf_unigrams_columns, tfidf_bigrams_columns, topics_columns, embeddings_columns], X_train_service, y_train_service, X_validation_service, y_validation_service))
print(test_model2(clf, [metadata_columns, syntax_columns, tfidf_unigrams_columns, tfidf_bigrams_columns, topics_columns, embeddings_columns], X_train_price, y_train_price, X_validation_price, y_validation_price))
print(test_model2(clf, [metadata_columns, syntax_columns, tfidf_unigrams_columns, tfidf_bigrams_columns, topics_columns, embeddings_columns], X_train_ambiance, y_train_ambiance, X_validation_ambiance, y_validation_ambiance))

In [ ]:
print(test_model2(clf, [metadata_columns, syntax_columns, tfidf_unigrams_columns, tfidf_bigrams_columns, topics_columns, embeddings_columns], X_train_food, y_train_food, X_test_food, y_test_food))
print(test_model2(clf, [metadata_columns, syntax_columns, tfidf_unigrams_columns, tfidf_bigrams_columns, topics_columns, embeddings_columns], X_train_service, y_train_service, X_test_service, y_test_service))
print(test_model2(clf, [metadata_columns, syntax_columns, tfidf_unigrams_columns, tfidf_bigrams_columns, topics_columns, embeddings_columns], X_train_price, y_train_price, X_test_price, y_test_price))
print(test_model2(clf, [metadata_columns, syntax_columns, tfidf_unigrams_columns, tfidf_bigrams_columns, topics_columns, embeddings_columns], X_train_ambiance, y_train_ambiance, X_test_ambiance, y_test_ambiance))

#### Average PR

In [ ]:
# precision
np.mean([precision_recall_fscore_support(y_test_food, food_p.predict(X_test_food[:, 1:]))[0][1],
         precision_recall_fscore_support(y_test_service, service_p.predict(X_test_service[:, 1:]))[0][1],
         precision_recall_fscore_support(y_test_price, price_p.predict(X_test_price[:, 1:]))[0][1],
         precision_recall_fscore_support(y_test_ambiance, ambiance_p.predict(X_test_ambiance[:, 1:]))[0][1]])

In [ ]:
# recall
np.mean([precision_recall_fscore_support(y_test_food, food_p.predict(X_test_food[:, 1:]))[1][1],
         precision_recall_fscore_support(y_test_service, service_p.predict(X_test_service[:, 1:]))[1][1],
         precision_recall_fscore_support(y_test_price, price_p.predict(X_test_price[:, 1:]))[1][1],
         precision_recall_fscore_support(y_test_ambiance, ambiance_p.predict(X_test_ambiance[:, 1:]))[1][1]])